In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:\\Users\\bhavy\\PycharmProjects\\FlaskProject4\\data\\sagefile.json').read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)

        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

79 documents
24 classes ['about', 'anxiety', 'death', 'depression', 'greeting', 'hate', 'hope', 'joke_friends', 'loneliness', 'meditation', 'mental_health_facts', 'no_approach', 'problem', 'scared', 'self_doubt', 'skills', 'sleep', 'stress', 'stupid', 'suicide', 'understand', 'user-advice', 'worthless', 'wrong']
140 unique lemmatized words ["'m", ',', '.', 'a', 'about', 'afraid', 'after', 'alone', 'am', 'an', 'and', 'another', 'anxious', 'approach', 'are', 'asleep', 'at', 'be', 'better', 'ca', 'calm', 'can', 'cheer', 'dealing', 'death', 'did', 'die', 'do', 'down', 'dumb', 'empty', 'end', 'enough', 'everything', 'fact', 'failure', 'falling', 'fearful', 'feel', 'foolish', 'get', 'giving', 'good', 'greeting', 'guide', 'guilty', 'handle', 'happens', 'happy', 'hate', 'hatred', 'have', 'having', 'health', 'hello', 'help', 'hey', 'hi', 'hope', 'how', 'i', 'important', 'in', 'is', 'issue', 'joke', 'just', 'know', 'laugh', 'laughter', 'life', 'like', 'listens', 'live', 'lonely', 'lost', 'made',

In [6]:
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

training = []

output_empty = [0] * len(classes)

for doc in documents:

    bag = []

    pattern_words = doc[0]

    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)


    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [7]:

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.keras', hist)
print("model created")

C:\Users\bhavy\PycharmProjects\FlaskProject4\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200


C:\Users\bhavy\PycharmProjects\FlaskProject4\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0360 - loss: 3.2083    
Epoch 2/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0498 - loss: 3.1671      
Epoch 3/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0305 - loss: 3.1469
Epoch 4/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1035 - loss: 3.0727     
Epoch 5/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1331 - loss: 3.0335     
Epoch 6/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1456 - loss: 2.9943 
Epoch 7/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1516 - loss: 2.9364     
Epoch 8/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2304 - loss: 2.8781 
Epoch 9/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2093 - loss: 2.7702     
Epoch 10/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3045 - loss: 2.6341 
Epoch 11/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3195 - loss: 2.3988 
Epoch 12/200
16/16 ━━━━━━━━━━━━━━━━━━━━ 0

In [8]:
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("model.keras")

# Load words and classes
with open("texts.pkl", "rb") as file:
    words = pickle.load(file)
with open("labels.pkl", "rb") as file:
    classes = pickle.load(file)

def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    return classes[np.argmax(res)]

def get_response(intent):
    for i in intents['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])

def chatbot_response(text):
    intent = predict_class(text)
    response = get_response(intent)
    return response

# Testing the chatbot

print(chatbot_response("i feel lonely"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
The seed beneath the earth feels darkness before it blooms into the most radiant flower. Trust in your own becoming.


In [17]:
print(chatbot_response("i have family issues"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
You are the traveler, and though the path may be obscured, each step is a step toward clarity. Keep walking, and you shall find your way.
